### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [3]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [4]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f"&fq={filter_query}&sort={sort}&fl={field_list}")


In [5]:
requests.get(query)
json.dumps(query, indent=4)

'"https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=suuaeNNpDxVS9NprauUOfpbsFkNWiD4P&begin_date=20130101&end_date=20230531&fq=section_name:\\"Movies\\" AND type_of_material:\\"Review\\" AND headline:\\"love\\"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count"'

In [6]:
# Create an empty list to store the reviews
nyt_movie_reviews = []

# loop through pages 0-19
for page in range(0,19):
    # create query with a page number
    # API results show 10 articles at a time
    query = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f"&fq={filter_query}&sort={sort}&fl={field_list}")
    query = f"{query}&page={str(page)}"
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
        # Print the page that was just retrieved
           nyt_movie_reviews.append(review)
        print(f"checked page {page}")

        # Print the page number that had no results then break from the loop
    except:
        print(f"No reviews found on page {page}")

checked page 0
checked page 1
checked page 2
checked page 3
checked page 4
checked page 5
checked page 6
checked page 7
checked page 8
checked page 9
checked page 10
checked page 11
checked page 12
checked page 13
checked page 14
checked page 15
checked page 16
checked page 17
checked page 18


In [7]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
nyt_movie_reviews
print(json.dumps(nyt_movie_reviews[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [8]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
nyt_movie_reviews_df = pd.json_normalize(nyt_movie_reviews)
nyt_movie_reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,https://www.nytimes.com/2017/06/22/movies/my-j...,"In “My Journey Through French Cinema,” Mr. Tav...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2017-06-22T21:14:56+0000,773,"Review: Those Movies, Himself — Bertrand Taver...",None,None,Reflecting on a Lifelong Love,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
186,https://www.nytimes.com/2017/06/22/movies/the-...,Kumail Nanjiani stars opposite Zoe Kazan in th...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-06-22T09:00:31+0000,1073,"Review: In ‘The Big Sick,’ Comedy Is Hard, Lov...",None,None,Love and (Almost) Death,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
187,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,"[{'name': 'creative_works', 'value': 'Lost in ...",2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
188,https://www.nytimes.com/2017/06/01/movies/vinc...,This combination crime drama and romance finds...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-06-01T19:52:51+0000,287,"Review: In ‘Vincent N Roxxy,’ Love Is in the A...",None,None,Vincent N Roxxy,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None


In [9]:
nyt_movie_reviews_df.to_csv("movie_reviews_NYT.csv", index = False)

In [10]:
nyt_movie_reviews_df["headline.main"].loc[5].split("\u2018")[1].split("\u2019")[0]


'One True Loves'

In [11]:
def column_correction(row):
    try: 
        return row.split("\u2018")[1].split("\u2019")[0]
    except:
         print(row)


In [12]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
nyt_movie_reviews_df["title"] = nyt_movie_reviews_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
nyt_movie_reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,https://www.nytimes.com/2017/06/22/movies/my-j...,"In “My Journey Through French Cinema,” Mr. Tav...",The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2017-06-22T21:14:56+0000,773,"Review: Those Movies, Himself — Bertrand Taver...",None,None,Reflecting on a Lifelong Love,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,"Review: Those Movies, Himself — Bertrand Taver..."
186,https://www.nytimes.com/2017/06/22/movies/the-...,Kumail Nanjiani stars opposite Zoe Kazan in th...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-06-22T09:00:31+0000,1073,"Review: In ‘The Big Sick,’ Comedy Is Hard, Lov...",None,None,Love and (Almost) Death,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,"The Big Sick,’ Comedy Is Hard, Love Harde"
187,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,"[{'name': 'creative_works', 'value': 'Lost in ...",2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None,Lost in Paris
188,https://www.nytimes.com/2017/06/01/movies/vinc...,This combination crime drama and romance finds...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-06-01T19:52:51+0000,287,"Review: In ‘Vincent N Roxxy,’ Love Is in the A...",None,None,Vincent N Roxxy,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,"Vincent N Roxxy

In [13]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
nyt_movie_reviews_df["keywords"] = nyt_movie_reviews_df["keywords"].apply(extract_keywords)
nyt_movie_reviews_df



,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,https://www.nytimes.com/2017/06/22/movies/my-j...,"In “My Journey Through French Cinema,” Mr. Tav...",The New York Times,subject: Documentary Films and Programs;person...,2017-06-22T21:14:56+0000,773,"Review: Those Movies, Himself — Bertrand Taver...",None,None,Reflecting on a Lifelong Love,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,"Review: Those Movies, Himself — Bertrand Taver..."
186,https://www.nytimes.com/2017/06/22/movies/the-...,Kumail Nanjiani stars opposite Zoe Kazan in th...,The New York Times,subject: Movies;creative_works: The Big Sick (...,2017-06-22T09:00:31+0000,1073,"Review: In ‘The Big Sick,’ Comedy Is Hard, Lov...",None,None,Love and (Almost) Death,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,"The Big Sick,’ Comedy Is Hard, Love Harde"
187,https://www.nytimes.com/2017/06/15/movies/lost...,The filmmakers Fiona Gordon and Dominique Abel...,The New York Times,creative_works: Lost in Paris (Movie);subject:...,2017-06-15T19:37:09+0000,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None,Lost in Paris
188,https://www.nytimes.com/2017/06/01/movies/vinc...,This combination crime drama and romance finds...,The New York Times,subject: Movies;creative_works: Vincent N Roxx...,2017-06-01T19:52:51+0000,287,"Review: In ‘Vincent N Roxxy,’ Love Is in the A...",None,None,Vincent N Roxxy,None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,"Vincent N Roxxy,’ Love Is in 

In [14]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = nyt_movie_reviews_df["title"].to_list()
titles

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [15]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key
query_url = url + tmdb_key_string
query_url

'https://api.themoviedb.org/3/search/movie?query=&api_key=4ff8f67448fef36fe986206281e94934'

In [16]:
# Create an empty list to store the results
TMDB_results = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        time.sleep(1)
        print(f"Sleeping at {request_counter} requests")

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    response = requests.get(url + title + tmdb_key_string)
    data = response.json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = data["results"][0]["id"]

        # Make a request for a the full movie details
        query_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
                    
        # Execute "GET" request with url
        data = requests.get(query_url).json()
        
        # Extract the genre names into a list
        genres = []
        for genre in data['genres']:
            genres.append(
                genre["name"]
            )

        # Extract the spoken_languages' English name into a list
        spoken_languages = []
        for language in data['spoken_languages']:
            spoken_languages.append(
                language["english_name"]
            )

        # Extract the production_countries' name into a list
        production_countries = []
        for country in data['production_countries']:
            production_countries.append(
                country["name"]
            )

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        TMDB_results.append(
            {
                "title": data['title'],
                "original_title": data['original_title'],
                "budget": data['budget'],
                "genre": genres,
                "spoken_languages": spoken_languages,
                "production_countries": production_countries
            }
        )
        
        # Print out the title that was found
        print(f"Found {title}")
    except:
        print(title + " not found.")

Found The Attachment Diaries
What’s Love Got to Do With It?’ Probably a Lo not found.
Found You Can Live Forever
Found A Tourist’s Guide to Love
Found Other People’s Children
Found One True Loves
Found The Lost Weekend: A Love Story
Found A Thousand and One
Found Your Place or Mine
Found Love in the Time of Fentanyl
Found Pamela, a Love Story
Found In From the Side
Found After Love
Found Alcarràs
Found Nelly & Nadine
Found Lady Chatterley’s Lover
Found The Sound of Christmas
Found The Inspection
Found Bones and All
Found My Policeman
Found About Fate
Found Waiting for Bojangles
Found I Love My Dad
Found A Love Song
Found Alone Together
Found Art of Love
Found The Wheel
Found Thor: Love and Thunder
Found Both Sides of the Blade
Found Fire of Love
Found Love & Gelato
Found Stay Prayed Up
Found Benediction
Found Dinner in America
Found In a New York Minute
Found Anaïs in Love
Found I Love America
Found See You Then
Found La Mami
Found Love After Love
Found Deep Water
Found Lucy and Desi
F

In [67]:
# Create an empty list to store the results
TMDB_results = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for i in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        time.sleep(1)
    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    requested_TMBD_DATA = requests.get(url + i + tmdb_key_string)
    data_TMDB = requested_TMBD_DATA.json()
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = data_TMDB["results"][0]["id"]
        # Make a request for a the full movie details
        full_details = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        
        # Execute "GET" request with url
        data_TMDB = requests.get(full_details).json()
        # Extract the genre names into a list
        genres = []
        for genre in data_TMDB["genres"]:
           genres.append(
               genre["name"])
        # Extract the spoken_languages' English name into a list
        English_translation = []
        #language_request = data_TMDB["original_language"]
        for langauge in data_TMDB["spoken_languages"]:
            English_translation.append(langauge["english_name"])
        # Extract the production_countries' name into a list
        production_countries = []
        for countries in data_TMDB["production_countries"]:
            production_countries.append(countries["name"])
        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        TMDB_results.append(
            {
                "title": data_TMDB["title"],
                "original Title": data_TMDB["original_title"],
                "budget": data_TMDB["budget"],
                "original_language": data_TMDB["original_language"],
                "homepage": data_TMDB["homepage"],
                "overview": data_TMDB["overview"],
                "popularity": data_TMDB["popularity"],
                "runtime": data_TMDB["runtime"], 
                "revenue": data_TMDB["revenue"],
                "release date": data_TMDB["release_date"],
                "vote average": data_TMDB["vote_average"],
                "vote count": data_TMDB["vote_count"],
                "genre": genres,
                "english Translation": English_translation,
                "production country": production_countries


            }
        )
        print(f"Found {i} in the data")
    except Exception as e:
        # Print out the title that was found
       print(e)
       print(f"The movie {i} was not found..... BIG SAD")
TMDB_results

Found The Attachment Diaries in the data
list index out of range
The movie What’s Love Got to Do With It?’ Probably a Lo was not found..... BIG SAD
Found You Can Live Forever in the data
Found A Tourist’s Guide to Love in the data
Found Other People’s Children in the data
Found One True Loves in the data
Found The Lost Weekend: A Love Story in the data
Found A Thousand and One in the data
Found Your Place or Mine in the data
Found Love in the Time of Fentanyl in the data
Found Pamela, a Love Story in the data
Found In From the Side in the data
Found After Love in the data
Found Alcarràs in the data
Found Nelly & Nadine in the data
Found Lady Chatterley’s Lover in the data
Found The Sound of Christmas in the data
Found The Inspection in the data
Found Bones and All in the data
Found My Policeman in the data
Found About Fate in the data
Found Waiting for Bojangles in the data
Found I Love My Dad in the data
Found A Love Song in the data
Found Alone Together in the data
Found Art of Love 

[{'title': 'The Attachment Diaries',
  'original Title': 'El apego',
  'budget': 0,
  'original_language': 'es',
  'homepage': '',
  'overview': 'Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.',
  'popularity': 1.4,
  'runtime': 102,
  'revenue': 0,
  'release date': '2021-10-07',
  'vote average': 4.0,
  'vote count': 2,
  'genre': ['Drama', 'Mystery', 'Thriller', 'Horror'],
  'english Translation': ['Spanish'],
  'production country': ['Argentina']},
 {'title': 'You Can Live Forever',
  'original Title': 'You Can Live Forever',
  'budget': 0,
  'original_language': 'en',
  'homepage': 'https://gooddeedentertainment.com/you-can-live-forever/',
  'ov

In [68]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
json.dumps(TMDB_results, indent=4)

'[\n    {\n        "title": "The Attachment Diaries",\n        "original Title": "El apego",\n        "budget": 0,\n        "original_language": "es",\n        "homepage": "",\n        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",\n        "popularity": 1.4,\n        "runtime": 102,\n        "revenue": 0,\n        "release date": "2021-10-07",\n        "vote average": 4.0,\n        "vote count": 2,\n        "genre": [\n            "Drama",\n            "Mystery",\n            "Thriller",\n            "Horror"\n        ],\n        "english Translation": [\n            "Spanish"\n        ],\n        "production country": [\n            

In [69]:
# Convert the results to a DataFrame
TMDB_DF = pd.json_normalize(TMDB_results)

### Merge and Clean the Data for Export

In [70]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(nyt_movie_reviews_df,TMDB_DF, on="title", how="inner")
merged_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release date,vote average,vote count,genre,english Translation,production country
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.400,102,0,2021-10-07,4.000,2,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",9.562,96,15055,2023-03-24,6.800,29,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,15.017,100,37820,2023-04-07,6.600,60,"[Romance, Drama, Comedy]",[English],[United States of America]
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,4.907,95,77145,2023-04-13,6.000,2,[Documentary],[English],[United States of America]
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,10.519,116,3395595,2023-03-31,7.000,74,"[Drama, Crime]","[Spanish, English, Portuguese]",[United States of America]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,https://www.nytimes.com/2018/01/11/movies/love...,"In Philippe Garrel’s drama, a 23-year-old woma...",The New York Times,subject: Movies;creative_works: Lover for a Da...,2018-01-11T16:20:53+0000,863,Review: A Man and a Woman (and Another Woman) ...,None,None,A Love Triangle With Angles That Are Awkward,...,"After a bad breakup, a college-aged Parisian m...",6.462,76,23692,2017-05-31,6.200,74,"[Drama, Romance]",[French],[France]
117,https://www.nytimes.com/2017/10/05/movies/the-...,Idris Elba and Kate Winslet crash into a mount...,The New York Times,subject: Movies;creative_works: The Mountain B...,2017-10-05T11:00:05+0000,272,Review: Love in a Cold Climate in ‘The Mountai...,None,None,The Mountain Between Us,...,Stranded on a mountain after a tragic plane cr...,28.597,109,62832209,2017-10-05,6.639,2256,"[Drama, Adventure, Romance]",[English],[United States of America]
118,https://www.nytimes.com/2017/09/14/movies/in-s...,A sheltered young woman enamored of the films ...,The New York Times,subject: Movies;creative_works: In Search of F...,2017-09-14T11:00:24+0000,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,...,"Lucy, a small-town girl from Ohio, discovers t...",4.934,93,0,2017-09-15,5.782,39,"[Drama, Adventure]",[English],[]
119,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies;creative_works: T

In [76]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
column_cleaning_list = []

# Create a list of characters to remove
undesirables = [ "[]", "[", "]", "\'", "\""]
 
def clean_column(s):
   for char in undesirables:
      s = s.replace(char, '')
   return s
# Loop through the list of columns to fix
for column in merged_df:
    # Convert the column to type 'str'
    if merged_df[column].astype(str).apply(lambda x: any(char in x for char in undesirables)).any():
      column_cleaning_list.append(column)
    # Loop through characters to remove
      merged_df[column] = merged_df[column].astype(str).apply(clean_column)
# Display the fixed DataFrame
merged_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release date,vote average,vote count,genre,english Translation,production country
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.4,102,0,2021-10-07,4.0,2,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",9.562,96,15055,2023-03-24,6.8,29,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,15.017,100,37820,2023-04-07,6.6,60,"Romance, Drama, Comedy",English,United States of America
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,4.907,95,77145,2023-04-13,6.0,2,Documentary,English,United States of America
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,10.519,116,3395595,2023-03-31,7.0,74,"Drama, Crime","Spanish, English, Portuguese",United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,https://www.nytimes.com/2018/01/11/movies/love...,"In Philippe Garrel’s drama, a 23-year-old woma...",The New York Times,subject: Movies;creative_works: Lover for a Da...,2018-01-11T16:20:53+0000,863,Review: A Man and a Woman (and Another Woman) ...,None,None,A Love Triangle With Angles That Are Awkward,...,"After a bad breakup, a college-aged Parisian m...",6.462,76,23692,2017-05-31,6.2,74,"Drama, Romance",French,France
117,https://www.nytimes.com/2017/10/05/movies/the-...,Idris Elba and Kate Winslet crash into a mount...,The New York Times,subject: Movies;creative_works: The Mountain B...,2017-10-05T11:00:05+0000,272,Review: Love in a Cold Climate in ‘The Mountai...,None,None,The Mountain Between Us,...,Stranded on a mountain after a tragic plane cr...,28.597,109,62832209,2017-10-05,6.639,2256,"Drama, Adventure, Romance",English,United States of America
118,https://www.nytimes.com/2017/09/14/movies/in-s...,A sheltered young woman enamored of the films ...,The New York Times,subject: Movies;creative_works: In Search of F...,2017-09-14T11:00:24+0000,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,...,"Lucy, a small-town girl from Ohio, discovers t...",4.934,93,0,2017-09-15,5.782,39,"Drama, Adventure",English,
119,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies;creative_works: The Last Face ...,2017-07-27T20:29:01+0000,280,Review: Aid Work

In [77]:
# Drop "byline.person" column
merged_df_clean = merged_df.drop(columns= ["byline.person"])

In [78]:
# Delete duplicate rows and reset index
merged_df = merged_df_clean.drop_duplicates()
merged_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release date,vote average,vote count,genre,english Translation,production country
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.4,102,0,2021-10-07,4.0,2,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",9.562,96,15055,2023-03-24,6.8,29,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,15.017,100,37820,2023-04-07,6.6,60,"Romance, Drama, Comedy",English,United States of America
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,4.907,95,77145,2023-04-13,6.0,2,Documentary,English,United States of America
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,10.519,116,3395595,2023-03-31,7.0,74,"Drama, Crime","Spanish, English, Portuguese",United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,https://www.nytimes.com/2018/01/11/movies/love...,"In Philippe Garrel’s drama, a 23-year-old woma...",The New York Times,subject: Movies;creative_works: Lover for a Da...,2018-01-11T16:20:53+0000,863,Review: A Man and a Woman (and Another Woman) ...,None,None,A Love Triangle With Angles That Are Awkward,...,"After a bad breakup, a college-aged Parisian m...",6.462,76,23692,2017-05-31,6.2,74,"Drama, Romance",French,France
117,https://www.nytimes.com/2017/10/05/movies/the-...,Idris Elba and Kate Winslet crash into a mount...,The New York Times,subject: Movies;creative_works: The Mountain B...,2017-10-05T11:00:05+0000,272,Review: Love in a Cold Climate in ‘The Mountai...,None,None,The Mountain Between Us,...,Stranded on a mountain after a tragic plane cr...,28.597,109,62832209,2017-10-05,6.639,2256,"Drama, Adventure, Romance",English,United States of America
118,https://www.nytimes.com/2017/09/14/movies/in-s...,A sheltered young woman enamored of the films ...,The New York Times,subject: Movies;creative_works: In Search of F...,2017-09-14T11:00:24+0000,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,...,"Lucy, a small-town girl from Ohio, discovers t...",4.934,93,0,2017-09-15,5.782,39,"Drama, Adventure",English,
119,https://www.nytimes.com/2017/07/27/movies/the-...,"In war-torn Liberia, Charlize Theron and Javie...",The New York Times,subject: Movies;creative_works: The Last Face ...,2017-07-27T20:29:01+0000,280,Review: Aid Work

In [79]:
# Export data to CSV without the index
TMDB_NYT_merged = merged_df
TMDB_NYT_merged.to_csv("TMDB_NYT_merged.csv", index=False)